<a href="https://colab.research.google.com/github/s528661/MyBERT_Practice/blob/main/BERT_MaskLanguageModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0
!pip install fugashi
!pip install ipadic

In [89]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from pytorch_transformers import BertTokenizer

In [90]:
import pandas as pd

In [91]:
data = pd.read_json("hf://datasets/stockmark/ner-wikipedia-dataset/ner.json")

In [92]:
text = data.iloc[-1]['text']
text = '[CLS] ' + text + ' [SEP]'
print(text)

[CLS] ダニス・タノヴィッチはユーゴスラビア出身の映画監督・脚本家である。 [SEP]


In [93]:
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [94]:
words = tokenizer.tokenize(text)

In [95]:
words

['[CLS]',
 'ダニ',
 '##ス',
 '##・',
 '##タ',
 '##ノ',
 '##ヴィッチ',
 'は',
 'ユーゴスラビア',
 '出身',
 'の',
 '映画',
 '監督',
 '・',
 '脚本',
 '家',
 'で',
 'ある',
 '。',
 '[SEP]']

In [96]:
msk_idx = 8
words[msk_idx] = "[MASK]"

In [97]:
words

['[CLS]',
 'ダニ',
 '##ス',
 '##・',
 '##タ',
 '##ノ',
 '##ヴィッチ',
 'は',
 '[MASK]',
 '出身',
 'の',
 '映画',
 '監督',
 '・',
 '脚本',
 '家',
 'で',
 'ある',
 '。',
 '[SEP]']

In [98]:
word_ids = tokenizer.convert_tokens_to_ids(words)               # word2token
word_tensor = torch.tensor([word_ids])                          # token2tensor

In [99]:
msk_model = BertForMaskedLM.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
msk_model.cuda()
msk_model.eval()

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [100]:
X = word_tensor.cuda()
y = msk_model(X)

In [101]:
result = y[0]
print(result.size())

torch.Size([1, 20, 32000])


In [102]:
_, max_ids = torch.topk(result[0][msk_idx], k=5)                                # Pick up Top 5 words

In [103]:
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())                # index2word
print(result_words)

['クロアチア', 'セルビア', 'チェコ', 'ポーランド', 'スロベニア']
